In [ ]:
!pip install lxml
!pip install beautifulsoup4

In [ ]:
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
import re

In [ ]:
def clean_from_html (s: str) -> str:
    bs_result = BeautifulSoup(s, 'lxml').text
    result = re.sub('\n', ' ', bs_result)
    return result

In [ ]:
df = pd.read_csv("/Users/tanyagolubev/Projects/deploy-impact-23-shift-3/data/glassdore/glassdoor.csv")

In [ ]:
df.head(20).T

In [ ]:
df.filter(
    items = [
    'header.jobTitle', 
    'header.employerName', 
    'header.location', 
    'job.description', 
    'job.jobTitleId', 
    'map.country',
    'map.employerName',
    'map.lat',
    'map.lng',
    'map.location',
    'map.address',
    'map.postalCode',
    'overview.industry',
    'overview.description',
    'overview.mission',
    'overview.website',
        ]
).T

In [ ]:
tech_keywords = [
    'computer',
    'vision',
    'developer',
    'data',
    'information',
    'software',
    'database',
    'programming',
    'ai',
    'ml',
    'application',
    'frontend',
    'backend'
          ]

senior_keywords = [
    'senior',
    'lead',
    'head',
    'partner'
]


In [ ]:
df['header.jobTitle'] = df['header.jobTitle'].map(lambda x: x.lower())
df['entry_IT_job'] = df['header.jobTitle'].str.contains('|'.join(tech_keywords))
df['entry_IT_job'] = ~df['header.jobTitle'].str.contains('|'.join(senior_keywords))
df['Swiss_based'] = df["map.country"].map(lambda x: x=="Switzerland" or x=="CH")
df_Swiss_IT  = df.loc[(df['Swiss_based'] ==True) & (df['entry_IT_job'] == True)]


In [ ]:
df_Swiss_IT['job.description_cleaned'] = df_Swiss_IT['job.description'].map(lambda x: clean_from_html(str(x)))

In [ ]:
df_Swiss_IT['job.description']

df_Swiss_IT['job.description']